In [4]:
import pandas as pd
import numpy as np
import textdistance

In [2]:
data_set = pd.read_csv("../data/interim/smiles_to_activity.csv")
data_set.head(10)

,canonical_smiles,standard_value,standard_units,pref_name
0,COc1cc(N)c(Cl)cc1C(=O)OCCN2CCN(CC2)c3ncccn3,4.5,nM,Serotonin 4 (5-HT4) receptor
1,COc1cc(N)c(Cl)cc1C(=O)OCCN2CCN(CC2)c3ncccn3,66.0,nM,Serotonin 4 (5-HT4) receptor
2,O=C1C=CC=C2[C@H]3CNC[C@H](C3)CN12,26000.0,nM,Neuronal acetylcholine receptor; alpha3/beta4
3,O=C1C=CC=C2[C@H]3CNC[C@H](C3)CN12,100000.0,nM,Acetylcholine receptor protein delta chain
4,CNCC\C=C\c1cccnc1,46000.0,nM,Neuronal acetylcholine receptor; alpha3/beta4
5,CN1CCC[C@H]1COc2ccc(Cl)nc2,1400.0,nM,Neuronal acetylcholine receptor; alpha4/beta2
6,CCc1cncc(OC[C@@H]2CCCN2C)c1,1000.0,nM,Neuronal acetylcholine receptor; alpha4/beta2
7,Clc1ccc(cn1)C2CC3CCC2N3,7.0,nM,Neuronal acetylcholine receptor; alpha3/beta4
8,Clc1ccc(cn1)C2CC3CCC2N3,200.0,nM,Acetylcholine receptor protein delta chain
9,C[C@H](CCC(=O)NCC(=O)O)[C@H]1CC[C@H]2[C@@H]3[C...,3000.0,nM,Bile acid receptor FXR


In [3]:
# Drop records with <10mM
data_set.drop(data_set[(data_set["standard_units"] == "mM") & (data_set["standard_value"] >= 10)].index, inplace=True)
data_set.drop(data_set[(data_set["standard_units"] == "uM") & (data_set["standard_value"] >= 10000)].index, inplace=True)
data_set.drop(data_set[(data_set["standard_units"] == "nM") & (data_set["standard_value"] >= 10000000)].index, inplace=True)

# Drop binding information as we are done with it
data_set.drop(["standard_value", "standard_units"], axis=1, inplace=True)

In [6]:
HAMMING_DISTANCE = 0.75
# Group targets by text similarity
name_groupings = pd.DataFrame({"pref_name": data_set["pref_name"].unique()})
name_groupings["group"] = -1
n_names = name_groupings.shape[0]
group = 0
for row in range(n_names):
    if name_groupings.loc[row, "group"] == -1:
        name_groupings.loc[row, "group"] = group
        group += 1
        for row2 in range(row, n_names):
            score = textdistance.hamming.normalized_similarity(name_groupings.loc[row, "pref_name"], name_groupings.loc[row2, "pref_name"])
            if score > HAMMING_DISTANCE:
                name_groupings.loc[row2, "group"] = name_groupings.loc[row, "group"]

In [23]:

data_set.head()

,canonical_smiles,pref_name
0,COc1cc(N)c(Cl)cc1C(=O)OCCN2CCN(CC2)c3ncccn3,"Serotonin 4 (5-HT4) receptor, Serotonin 7 (5-H..."
1,COc1cc(N)c(Cl)cc1C(=O)OCCN2CCN(CC2)c3ncccn3,"Serotonin 4 (5-HT4) receptor, Serotonin 7 (5-H..."
2,O=C1C=CC=C2[C@H]3CNC[C@H](C3)CN12,"Neuronal acetylcholine receptor; alpha3/beta4,..."
3,O=C1C=CC=C2[C@H]3CNC[C@H](C3)CN12,"Acetylcholine receptor protein delta chain, Ac..."
4,CNCC\C=C\c1cccnc1,"Neuronal acetylcholine receptor; alpha3/beta4,..."
